<div>
    <table>
        <tr>
            <td>
                <center>
                    <h1>Wurst Introduction</h1>
                     <a href="https://www.psi.ch/en/ta/people/romain-sacchi">Karin Treyer</a> (PSI)
                    <br><br>
                    Duration: 1 hour 30 minutes.
                </center>
            </td>
        </tr>
    </div>

Github repository: [https://github.com/polca/wurst](https://github.com/polca/wurst)

Documentation: [https://wurst.readthedocs.io/](https://wurst.readthedocs.io/)

Wurst authors: Chris Mutel, Brian Cox

<div class="alert alert-info">
Note: we will be using <a href="https://docs.brightway.dev/en/latest/content/installation/index.html">Brightway 2.5</a>, not <a href="https://docs.brightway.dev/en/legacy/">Brightway 2</a>. From the user end side, very little differ between the two. The code executed throughout this notebook works with both versions... probably.
</div>


### Scenario: A world without fossil fuels

#### Background:
By 2050, fossil fuels in power generation are phased out due to stringent climate policies and technological advancements in renewables.


#### Objective:

1. Replace the contribution of fossil-based technologies in electricity markets with non-hydro renewables throughout the database.
The former share of fossil-based electricity should be redistributed evenly across already supplying renewables.

2. Compare the electricity markets, at low-voltage level with respect to Climate Change, Fine Particle emissions and Abiotic Resource Depletion. Can you identify some trade-offs?

### 1. Import:

Load the necessary libraries (`wurst`, `bw2data`, and maybe a plotting library like `matplotlib`).

In [ ]:
import wurst
import wurst.searching as ws
import bw2io, bw2data
import matplotlib.pyplot as plt
from prettytable import PrettyTable

Select/activate a `bw2data` project where you have an ecoinvent database.

Using `wurst`, extract the ecoinvent database to turn it into a list of dictionaries.
Or use the `load_db()` function from the tutorial to unpickle the ecoinvent database instead, as it is faster.

Build `wurst` filters to select only markets for electricity (high, medium, and low voltage). Filter out datasets specific to industries (coal, aluminium, railways, etc.) or for certified renewable electricity.

In [ ]:
market_filters = [
    ws.contains("name", "market for electricity"),
    ws.equals("unit", "kilowatt hour"),
    ws.exclude(ws.either(
        *[
            ws.contains("name", name) for name in [
                "product",
                "generic",
                "aluminium",
                "reuse",
                "industry",
                "Railways",
                "mining"
            ]
        ]
    )),
]

Iterate through the database using the filters you built above. Inside each dataset, iterate over the technosphere exchanges. You may need to ignore exchanges that represent market losses, for example. Try to distinguish suppliers of fossil-based electricity from renewable ones.

In [ ]:
technosphere_filters = [
    ws.equals("unit", "kilowatt hour"),
    ws.exclude(ws.contains("name", "market for")),
    ws.exclude(ws.contains("name", "voltage")),
]

In [ ]:
fossil_suppliers = [
    "coal", "gas", "lignite", "oil", "peat"
]
renewable_suppliers = [
    "photovoltaic", "wind"
]

In [ ]:
modifications = []
countries_wo_renew = []

for ds in ws.get_many(
    db,
    *market_filters
):
    # first, we check if there are renewable electricity suppliers
    number_renewables = len([x for x in ws.technosphere(ds, *technosphere_filters) if any(y in x["name"] for y in renewable_suppliers)])
    # we also need to check if there are fossil electricity suppliers
    number_fossil = len([x for x in ws.technosphere(ds, *technosphere_filters) if any(y in x["name"] for y in fossil_suppliers)])

    
    if number_fossil > 0 and number_renewables > 0:
        shares = {}
        # we iterate a first time to collect current shares
        for techno in ws.technosphere(
            ds,
            *technosphere_filters
            ):

            # fill in here the code to collect shares
            
    
        # we iterate a second time to zero-out fossil-based electricity
        surplus = 0
        for techno in ws.technosphere(
            ds,
            *technosphere_filters
            ):
            
            if any(x in techno["name"] for x in fossil_suppliers):
                modifications.append(
                    [ds["name"].split(", ")[-1], ds["location"], techno["name"][:30], "{:.2f}".format(techno["amount"]), 0]
                )

        # fill in here the code to set fossil-based suppliers to zero
                
    
        # we iterate a third time to distribute the surplus
        
        for techno in ws.technosphere(
            ds,
            *technosphere_filters
            ):
            if any(x in techno["name"] for x in renewable_suppliers):
                modifications.append(
                    [ds["name"].split(", ")[-1], ds["location"], techno["name"][:30], "{:.2f}".format(techno["amount"]), "{:.2f}".format(new_share)]
                )

                # fill in here the code to redistribute the fossil-based mix share to renewables-based suppliers
                

table = PrettyTable(["Name", "Loc", "Tech", "Old share", "New share",])
table.add_rows(modifications)
print(table)

In [ ]:
unique_countries = list(set([m[1] for m in modifications]))

Export the database back to the `brightway` project.

Build a list of functional units to compare the new electricity markets with the original ones. Pick only those we have modified (use `unique_countries`).

In [ ]:
methods = [
    ('IPCC 2021', 'climate change: fossil', 'global warming potential (GWP100)'),
    ('Inventory results and indicators', 'emissions to air', 'particulate matter, < 2.5 um'),
    ('EF v3.1', 'material resources: metals/minerals', 'abiotic depletion potential (ADP): elements (ultimate reserves)'),
]

In [ ]:
methods_units = {
    m: bw2data.Method(m).metadata["unit"]
    for m in methods
}
methods_units

In [ ]:
import bw2calc
import numpy as np
from bw2calc.errors import OutsideTechnosphere

# we create an array to store our results
results = []

# we initialize the LCA matrices
lca = bw2calc.LCA({FU[0]: 1}, method)
lca.lci(factorize=True)

for i, fu in enumerate(FU):
    try:
        lca.lci({fu.id: 1})
    except OutsideTechnosphere:
        lca = bw2calc.LCA({fu: 1}, method)
        lca.lci(factorize=True)
        
    for method in methods:
        lca.switch_method(method)
        lca.lcia()
        results.append([fu["location"], fu["database"], str(method), lca.score])

Format the results into a pandas dataframe, with columns `name`, `database`, `method` and `score`.

In [ ]:
import pandas as pd



In [ ]:
df = df.sort_values(by=["name", "database", "method"])

In [ ]:
df

In [ ]:
import seaborn as sns

for method in methods:
    plt.figure(figsize=(15,6))
    # plot with seaborn barplot
    sns.barplot(data=df.loc[df["method"]==str(method)], x='name', y='score', hue='database')
    
    plt.xticks()
    plt.ylabel(f"{methods_units[method]}/kWh")
    plt.title(method)